# ДЗ 17: Seq2Seq и Seq2Seq + Attention (английский → русский)

Кратко:

* **датасет перевода с английского на русский** на Hugging Face: `Helsinki-NLP/opus-100` (`en` / `ru`),
* обучаем общий SentencePiece‑токенизатор,
* строим две модели:
  * Seq2Seq на GRU,
  * Seq2Seq + Bahdanau Attention,
* обучаем обе модели на тренировочной выборке,
* переводим 30 предложений из тестовой выборки обеими моделями и сравниваем результаты.

Особенности реализации:

* учёт паддингов через `pack_padded_sequence` / `pad_packed_sequence` в энкодере,
* 2 слоя GRU + `dropout`,
* shared embeddings + weight tying,
* label smoothing, gradient clipping,
* план по teacher forcing (1.0 → 0.6),
* градиентная аккумуляция и AMP (FP16) для работы для работы в уловиях умеренного объёма доступной памяти GPU 16(Гб).


In [ ]:
import math
import random
from pathlib import Path
from contextlib import nullcontext

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import sentencepiece as spm
from tqdm.auto import tqdm

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE



## Корпус текстов

In [ ]:

pair = "en-ru"
LANG_SRC, LANG_TGT = pair.split("-")
ds = load_dataset("Helsinki-NLP/opus-100", pair)
print(ds)
print(ds["train"][0]["translation"])

ds_train = ds["train"]
ds_valid   = ds["validation"] if "validation" in ds else ds.get("dev", None)
ds_test  = ds["test"] if "test" in ds else None
if ds_test is None:
    tmp = ds_train.train_test_split(test_size=0.05, seed=SEED)
    ds_train, ds_test = tmp["train"], tmp["test"]
if ds_valid is None:
    tmp = ds_train.train_test_split(test_size=0.05, seed=SEED)
    ds_train, ds_valid = tmp["train"], tmp["test"]
    
len(ds_train), len(ds_valid), len(ds_test)



DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})
{'en': "Yeah, that's not exactly...", 'ru': 'Да, но не совсем...'}


(1000000, 2000, 2000)

## Обучение токенизатора

In [ ]:
VOCAB_SIZE = 16000 # Размер словаря
SPM_MODEL_PREFIX = "spm_bpe_opus100_en_ru_16k_2"
SPM_MODEL_FILE = f"{SPM_MODEL_PREFIX}.model"

if not Path(SPM_MODEL_FILE).exists():
    corpus_path = Path("spm_corpus_opus100_en_ru_2.txt")
    MAX_LINES = 1_000_000  # максимум пар предложений для корпуса SPM

    with corpus_path.open("w", encoding="utf-8") as f:
        for i, ex in enumerate(tqdm(ds_train, desc="SPM corpus (OPUS-100)")):
            if i >= MAX_LINES:
                break
            tr = ex["translation"]
            src_text = tr[LANG_SRC].replace("\n", " ")
            tgt_text = tr[LANG_TGT].replace("\n", " ")
            f.write(src_text + "\n")
            f.write(tgt_text + "\n")

    spm.SentencePieceTrainer.train(
        input=str(corpus_path),
        model_prefix=SPM_MODEL_PREFIX,
        vocab_size=VOCAB_SIZE,
        model_type="bpe", # Byte Pair Encoding (BPE) — это метод постепенного слияния наиболее часто встречающихся пар байтов (или символов) в более длинные токены.  Слова, которых не было в обучающей выборке, можно разбить на известные подслова.
        character_coverage=0.9995,
        pad_id=0,
        unk_id=1,
        bos_id=2,
        eos_id=3,
        input_sentence_size=1_000_000, # был взят весь миллион предложений
        shuffle_input_sentence=True,
    )
    print("SentencePiece модель обучена.")
else:
    print("SentencePiece модель загружена.")

sp = spm.SentencePieceProcessor()
sp.load(SPM_MODEL_FILE)

PAD_ID = sp.pad_id()
UNK_ID = sp.unk_id()
BOS_ID = sp.bos_id()
EOS_ID = sp.eos_id()
VOCAB_SIZE = sp.vocab_size()

PAD_ID, UNK_ID, BOS_ID, EOS_ID, VOCAB_SIZE

SentencePiece модель загружена.


(0, 1, 2, 3, 16000)

## Токенизация и создание батчей

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

MAX_LEN = 80  # максимум токенов SPM (включая BOS/EOS)

def encode_text(text: str, max_len: int = MAX_LEN,
                add_bos: bool = True, add_eos: bool = True):
    """
    Преобразует текст в последовательность идентификаторов токенов и возвращает её в виде тензора PyTorch.

    Функция выполняет токенизацию входного текста с использованием внешнего токенизатора `SentencePiece`,
    добавляет специальные токены начала (BOS) и конца (EOS) последовательности при необходимости,
    обрезает результат до заданной максимальной длины и преобразует в тензор.
    """
    
    ids = sp.encode(text, out_type=int)
    if add_bos:
        ids = [BOS_ID] + ids
    if add_eos:
        ids = ids + [EOS_ID]
    if len(ids) > max_len:
        ids = ids[:max_len]
    return torch.tensor(ids, dtype=torch.long)


class TranslationDataset(Dataset):
    def __init__(self, hf_split, lang_src: str, lang_tgt: str, max_len: int = MAX_LEN):
        self.data = hf_split
        self.lang_src = lang_src
        self.lang_tgt = lang_tgt
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Извлекает и кодирует пару текстов по заданному индексу.
        """
        tr = self.data[idx]["translation"]
        src_text = tr[self.lang_src]
        tgt_text = tr[self.lang_tgt]
        src = encode_text(src_text, max_len=self.max_len)
        tgt = encode_text(tgt_text, max_len=self.max_len)
        return src, tgt


def pad_sequence_sp(seqs):
    """
    Дополняет последовательности до одинаковой длины, создавая батч для обработки в PyTorch.

    Функция принимает список тензоров разной длины, определяет максимальную длину,
    и дополняет более короткие последовательности значением PAD_ID до этой длины,
    формируя двумерный тензор с равномерными размерами.
    """
    lens = [len(s) for s in seqs]
    max_len = max(lens)
    out = torch.full((len(seqs), max_len), PAD_ID, dtype=torch.long)
    for i, s in enumerate(seqs):
        out[i, : len(s)] = s
    return out


def collate_fn_sp(batch):
    """
    Функция для объединения списка образцов в батч с использованием дополнения (padding).

    Используется как collate_fn в DataLoader PyTorch. Принимает список пар тензоров
    (источник, цель), разделяет их на отдельные списки, выравнивает последовательности
    по максимальной длине с помощью pad_sequence_sp и возвращает батчи с соответствующими длинами.
    """
    src_list, tgt_list = zip(*batch)
    src_pad = pad_sequence_sp(src_list)   # форма [B,S]  B — размер батча, S — максимальная длина в батче для источника, T — для цели
    tgt_pad = pad_sequence_sp(tgt_list)   # форма [B,T]
    src_len = (src_pad != PAD_ID).sum(dim=1)  # [B]
    return src_pad, tgt_pad, src_len


train_dataset = TranslationDataset(ds_train, LANG_SRC, LANG_TGT)
valid_dataset = TranslationDataset(ds_valid, LANG_SRC, LANG_TGT)
test_dataset  = TranslationDataset(ds_test,  LANG_SRC, LANG_TGT)

BATCH_SIZE = 192
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate_fn_sp, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False,
                          collate_fn=collate_fn_sp, num_workers=0, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False,
                          collate_fn=collate_fn_sp, num_workers=0, pin_memory=True)

next(iter(train_loader))[0].shape


torch.Size([192, 80])

## Определение нейросетевых моделей seq2seq (энкодер-декор и энкодер-декор-внимание)

### Определение классов энкодера, декодера, энкодер+внимание, декодер+внимание

In [ ]:

class Encoder(nn.Module):
    """
    Модуль энкодера на основе GRU.

    Энкодер преобразует входную последовательность токенов в скрытые состояния с использованием
    эмбеддингов и многослойной GRU-сети. Поддерживает обработку паддированных последовательностей
    с помощью pack_padded_sequence для эффективного обучения.

    embedding : nn.Embedding
        Слой эмбеддингов, преобразующий индексы токенов в векторные представления.
        Использует PAD_ID как индекс для игнорирования паддинг-токенов.
    dropout : nn.Dropout
        Слой дропаута для регуляризации, применяется к эмбеддингам.
    gru : nn.GRU
        Многослойная рекуррентная сеть GRU.
        При наличии более чем одного слоя применяется дропаут между слоями.
    """
    def __init__(self, vocab_size, emb_dim, hid_dim, num_layers=2, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD_ID)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(
            emb_dim,
            hid_dim,
            batch_first=True,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
        )

    def forward(self, src, lengths=None):
        # src: [B,S]
        emb = self.dropout(self.embedding(src))  # [B,S,E]
        if lengths is not None:
            packed = nn.utils.rnn.pack_padded_sequence(
                emb,
                lengths.cpu(),
                batch_first=True,
                enforce_sorted=False,
            )
            packed_out, hidden = self.gru(packed)
            outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        else:
            outputs, hidden = self.gru(emb)
        # outputs: [B,S,H], hidden: [L,B,H]
        return outputs, hidden


class Decoder(nn.Module):
    """
    Модуль декодера на основе GRU для задач генерации последовательностей, таких как машинный перевод.

    Декодер принимает на вход предыдущий токен и скрытое состояние из энкодера, обрабатывает его
    с помощью слоя эмбеддингов и многослойной GRU-сети, а затем выдаёт логиты для предсказания следующего токена.

    embedding : nn.Embedding
        Слой эмбеддингов, преобразующий индексы токенов в векторные представления.
        Использует PAD_ID как индекс для игнорирования паддинг-токенов.
    fc_out : nn.Linear
        Линейный слой, преобразующий выход GRU в логиты по всему словарю, используется для предсказания следующего токена.
    """
    def __init__(self, vocab_size, emb_dim, hid_dim, num_layers=2, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD_ID)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(
            emb_dim,
            hid_dim,
            batch_first=True,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.fc_out = nn.Linear(hid_dim, vocab_size, bias=True)

    def forward(self, input_tok, hidden):
        # input_tok: [B]
        emb = self.dropout(self.embedding(input_tok.unsqueeze(1)))  # [B,1,E]
        out, hidden = self.gru(emb, hidden)
        logits = self.fc_out(out.squeeze(1))  # [B,V]
        return logits, hidden


class BahdanauAttention(nn.Module):
    """
    Модуль механизма внимания Бахданау для рекуррентных моделей.

    Реализует аддитивное внимание, которое позволяет декодеру
    сосредотачиваться на различных частях входной последовательности при генерации каждого токена.
    """
    def __init__(self, hid_dim):
        super().__init__()
        self.W1 = nn.Linear(hid_dim, hid_dim)
        self.W2 = nn.Linear(hid_dim, hid_dim)
        self.v = nn.Linear(hid_dim, 1)

    def forward(self, hidden, enc_outs, mask=None):
        # hidden: [L,B,H] → берём последний слой
        h = hidden[-1]  # [B,H]
        # enc_outs: [B,S,H]
        score = self.v(torch.tanh(self.W1(enc_outs) + self.W2(h).unsqueeze(1)))  # [B,S,1]
        attn = torch.softmax(score, dim=1)  # [B,S,1]
        if mask is not None:
            attn = attn * mask.unsqueeze(-1)
            attn = attn / (attn.sum(dim=1, keepdim=True) + 1e-9)
        ctx = (attn * enc_outs).sum(dim=1)  # [B,H]
        return ctx, attn.squeeze(-1)        # [B,H], [B,S]


class AttnDecoder(nn.Module):
    """
    Декодер с механизмом внимания Бахданау.

    Этот модуль расширяет стандартный GRU-декодер, добавляя механизм внимания,
    который позволяет модели динамически фокусироваться на различных частях
    входной последовательности при генерации каждого токена. Объединяет эмбеддинг
    входного токена и контекстный вектор (на основе внимания) перед подачей в GRU.
    """
    def __init__(self, vocab_size, emb_dim, hid_dim, num_layers=2, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD_ID)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(
            emb_dim + hid_dim,
            hid_dim,
            batch_first=True,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.fc_out = nn.Linear(hid_dim, vocab_size, bias=True)
        self.attn = BahdanauAttention(hid_dim)

    def forward(self, input_tok, hidden, enc_outs, src_mask=None):
        emb = self.dropout(self.embedding(input_tok.unsqueeze(1)))  # [B,1,E]
        ctx, _ = self.attn(hidden, enc_outs, mask=src_mask)         # [B,H]
        x = torch.cat([emb, ctx.unsqueeze(1)], dim=-1)              # [B,1,E+H]
        out, hidden = self.gru(x, hidden)
        logits = self.fc_out(out.squeeze(1))
        return logits, hidden


### Определение моделей

In [ ]:

def make_src_mask(src):
    return (src != PAD_ID)  # [B,S], bool


class Seq2Seq(nn.Module):
    """
    Модель Seq2Seq.

    Архитектура объединяет энкодер и декодер.
    Поддерживает режим обучения с параметром `tf_ratio` -- Вероятность использования teacher forcing (значение от 0 до 1). При значении 1 — всегда использует истинные токены, при 0 — только предсказания модели.
    """
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt, lengths=None, tf_ratio: float = 1.0):
        enc_outs, hidden = self.encoder(src, lengths)
        input_tok = tgt[:, 0]   # BOS
        outs = []
        for t in range(1, tgt.size(1)):
            logits, hidden = self.decoder(input_tok, hidden)
            outs.append(logits.unsqueeze(1))
            with torch.no_grad():
                use_tf = torch.rand(input_tok.size(0), device=tgt.device) < tf_ratio
                greedy = logits.argmax(dim=-1)
                next_tok = torch.where(use_tf, tgt[:, t], greedy)
            input_tok = next_tok
        return torch.cat(outs, dim=1)  # [B,T-1,V]


class Seq2SeqAttn(nn.Module):
    """
    Модель последовательность-в-последовательность с механизмом внимания.

    Архитектура объединяет энкодер и декодер с поддержкой внимания, что позволяет
    декодеру динамически обращаться к различным частям входной последовательности
    при генерации каждого токена выходной последовательности.
    
    Выполняет кодирование входной последовательности и последовательную генерацию выхода с использованием teacher forcing (tf_ratio) и механизма внимания.
    На каждом шаге следующий входной токен выбирается: с вероятностью `tf_ratio` — из целевой последовательности (истина), иначе — жадное предсказание модели (inference).
    Это позволяет комбинировать обучение с учителем и автономную генерацию.
    """
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt, src_mask=None, lengths=None, tf_ratio: float = 1.0):
        enc_outs, hidden = self.encoder(src, lengths)
        input_tok = tgt[:, 0]
        outs = []
        for t in range(1, tgt.size(1)):
            logits, hidden = self.decoder(input_tok, hidden, enc_outs, src_mask)
            outs.append(logits.unsqueeze(1))
            with torch.no_grad():
                use_tf = torch.rand(input_tok.size(0), device=tgt.device) < tf_ratio
                greedy = logits.argmax(dim=-1)
                next_tok = torch.where(use_tf, tgt[:, t], greedy)
            input_tok = next_tok
        return torch.cat(outs, dim=1)


def share_and_tie(enc: Encoder, dec: nn.Module):
    dec.embedding.weight = enc.embedding.weight
    if hasattr(dec, "fc_out"):
        dec.fc_out.weight = dec.embedding.weight

## Обучение моделей

In [ ]:

EPOCHS = 25
EMB_DIM = 512
HID_DIM = 512
LR = 1e-3
WEIGHT_DECAY = 1e-4
ACCUM_STEPS = 2  # эффективный батч ≈ BATCH_SIZE * ACCUM_STEPS

def build_seq2seq():
    enc = Encoder(VOCAB_SIZE, EMB_DIM, HID_DIM, num_layers=2, dropout=0.2)
    dec = Decoder(VOCAB_SIZE, EMB_DIM, HID_DIM, num_layers=2, dropout=0.2)
    share_and_tie(enc, dec)
    return Seq2Seq(enc, dec)

def build_seq2seq_attn():
    enc = Encoder(VOCAB_SIZE, EMB_DIM, HID_DIM, num_layers=2, dropout=0.2)
    dec = AttnDecoder(VOCAB_SIZE, EMB_DIM, HID_DIM, num_layers=2, dropout=0.2)
    share_and_tie(enc, dec)
    return Seq2SeqAttn(enc, dec)


model_s2s = build_seq2seq().to(DEVICE)
model_attn = build_seq2seq_attn().to(DEVICE)

opt_s2s = torch.optim.AdamW(model_s2s.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, betas=(0.9, 0.98))
opt_attn = torch.optim.AdamW(model_attn.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, betas=(0.9, 0.98))

sched_s2s = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt_s2s, mode="min", factor=0.5, patience=2, verbose=True
)
sched_attn = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt_attn, mode="min", factor=0.5, patience=2, verbose=True
)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID, label_smoothing=0.1)


def tf_schedule(epoch: int):
    # teacher forcing: 1.0 → 0.6
    return max(0.6, 1.0 - 0.02 * (epoch - 1))


def train_epoch(model, loader, optimizer, criterion, clip=1.0, tf_ratio=1.0, accum_steps=2):
    model.train()
    total = 0.0
    scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))
    autocast_ctx = torch.cuda.amp.autocast if DEVICE.type == "cuda" else nullcontext

    optimizer.zero_grad(set_to_none=True)
    for step, (src, tgt, src_len) in enumerate(tqdm(loader, desc="train", leave=False), start=1):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        src_len = src_len.to(DEVICE)

        with autocast_ctx(dtype=torch.float16 if DEVICE.type == "cuda" else None):
            if isinstance(model, Seq2SeqAttn):
                logits = model(src, tgt,
                               src_mask=make_src_mask(src),
                               lengths=src_len,
                               tf_ratio=tf_ratio)
            else:
                logits = model(src, tgt, lengths=src_len, tf_ratio=tf_ratio)
            loss = criterion(
                logits.reshape(-1, logits.size(-1)),
                tgt[:, 1:].reshape(-1)
            )
            loss = loss / accum_steps

        scaler.scale(loss).backward()

        if step % accum_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

        total += loss.item() * accum_steps

    if DEVICE.type == "cuda":
        torch.cuda.empty_cache()
    return total / max(1, len(loader))


@torch.no_grad()
def eval_epoch(model, loader, criterion):
    model.eval()
    total = 0.0
    autocast_ctx = torch.cuda.amp.autocast if DEVICE.type == "cuda" else nullcontext

    for src, tgt, src_len in tqdm(loader, desc="valid", leave=False):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        src_len = src_len.to(DEVICE)

        with autocast_ctx(dtype=torch.float16 if DEVICE.type == "cuda" else None):
            if isinstance(model, Seq2SeqAttn):
                logits = model(src, tgt,
                               src_mask=make_src_mask(src),
                               lengths=src_len,
                               tf_ratio=1.0)
            else:
                logits = model(src, tgt, lengths=src_len, tf_ratio=1.0)
            loss = criterion(
                logits.reshape(-1, logits.size(-1)),
                tgt[:, 1:].reshape(-1)
            )
        total += loss.item()

    if DEVICE.type == "cuda":
        torch.cuda.empty_cache()
    return total / max(1, len(loader))


def train_and_save(model, optimizer, scheduler, model_path, label, history_list):
    best = float("inf")
    bad = 0
    patience = 6
    start_epoch = len(history_list)
    print(f"{label}: обучение начинается с эпохи {start_epoch + 1}")

    for k in range(1, EPOCHS + 1):
        epoch = start_epoch + k
        tf_ratio = tf_schedule(epoch)

        train_loss = train_epoch(
            model, train_loader, optimizer, criterion,
            clip=1.0, tf_ratio=tf_ratio, accum_steps=ACCUM_STEPS
        )
        val_loss = eval_epoch(model, valid_loader, criterion)
        scheduler.step(val_loss)

        history_list.append((train_loss, val_loss))
        print(f"[{label}] Эпоха {epoch:02d} | tf={tf_ratio:.2f} | "
              f"train={train_loss:.3f} | val={val_loss:.3f}")

        if val_loss + 1e-4 < best:
            best = val_loss
            bad = 0
            torch.save(model.state_dict(), model_path)
        else:
            bad += 1
            if bad >= patience:
                print(f"{label}: ранняя остановка на эпохе {epoch:02d}")
                break

    print(f"{label}: лучшие веса сохранены → {model_path}")


f:\Staff\Иннополис\Innopolis\.venv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:

history_s2s = []
history_attn = []

MODEL_S2S_PATH = "model_seq2seq_en_ru.pt"
MODEL_ATT_PATH = "model_seq2seq_attn_en_ru.pt"

# Запуск обучения:
#train_and_save(model_s2s, opt_s2s, sched_s2s, MODEL_S2S_PATH, "Seq2Seq", history_s2s)
#train_and_save(model_attn, opt_attn, sched_attn, MODEL_ATT_PATH, "Seq2Seq+Attn", history_attn)


## Инференс моделей на 30 примерах

In [ ]:

@torch.no_grad()
def decode_ids(ids):
    clean = []
    for i in ids:
        if i in (PAD_ID, BOS_ID):
            continue
        if i == EOS_ID:
            break
        clean.append(i)
    return sp.decode(clean)


@torch.no_grad()
def greedy_translate_s2s(model, src_batch, src_len):
    model.eval()
    src_batch = src_batch.to(DEVICE)
    src_len = src_len.to(DEVICE)

    enc_outs, hidden = model.encoder(src_batch, src_len)
    B = src_batch.size(0)
    input_tok = torch.full((B,), BOS_ID, dtype=torch.long, device=DEVICE)

    finished = torch.zeros(B, dtype=torch.bool, device=DEVICE)
    outputs = [[] for _ in range(B)]

    for _ in range(MAX_LEN):
        logits, hidden = model.decoder(input_tok, hidden)
        next_tok = logits.argmax(dim=-1)
        for i in range(B):
            if not finished[i]:
                outputs[i].append(next_tok[i].item())
                if next_tok[i].item() == EOS_ID:
                    finished[i] = True
        if finished.all():
            break
        input_tok = next_tok

    return [decode_ids(seq) for seq in outputs]


@torch.no_grad()
def greedy_translate_attn(model, src_batch, src_len):
    model.eval()
    src_batch = src_batch.to(DEVICE)
    src_len = src_len.to(DEVICE)
    src_mask = make_src_mask(src_batch)

    enc_outs, hidden = model.encoder(src_batch, src_len)
    B = src_batch.size(0)
    input_tok = torch.full((B,), BOS_ID, dtype=torch.long, device=DEVICE)

    finished = torch.zeros(B, dtype=torch.bool, device=DEVICE)
    outputs = [[] for _ in range(B)]

    for _ in range(MAX_LEN):
        logits, hidden = model.decoder(input_tok, hidden, enc_outs, src_mask)
        next_tok = logits.argmax(dim=-1)
        for i in range(B):
            if not finished[i]:
                outputs[i].append(next_tok[i].item())
                if next_tok[i].item() == EOS_ID:
                    finished[i] = True
        if finished.all():
            break
        input_tok = next_tok

    return [decode_ids(seq) for seq in outputs]


# Загрузка лучших весов после обучения (раскомментировать при наличии файлов):
model_s2s.load_state_dict(torch.load(MODEL_S2S_PATH, map_location=DEVICE))
model_attn.load_state_dict(torch.load(MODEL_ATT_PATH, map_location=DEVICE))


def sample_test_batch(n=30):
    subset = [ds_test[i] for i in range(min(n, len(ds_test)))]
    src_list = []
    tgt_text = []
    for ex in subset:
        tr = ex["translation"]
        src_list.append(encode_text(tr[LANG_SRC]))  # or tr[SRC_LANG]
        tgt_text.append(tr[LANG_TGT])  # or tr[TGT_LANG]
    src_pad = pad_sequence_sp(src_list)
    src_len = (src_pad != PAD_ID).sum(dim=1)
    return src_pad, src_len, tgt_text


src_pad, src_len, tgt_gold = sample_test_batch(30)
pred_s2s = greedy_translate_s2s(model_s2s, src_pad, src_len)
pred_att = greedy_translate_attn(model_attn, src_pad, src_len)

for i in range(len(tgt_gold)):
    print(f"=== Пример {i+1} ===")
    print("SRC:", decode_ids(src_pad[i].tolist()))
    print("REF:", tgt_gold[i])
    print("S2S:", pred_s2s[i])
    print("ATT:", pred_att[i])
    print()


C:\Users\stepa\AppData\Local\Temp\ipykernel_31864\3802357344.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_s2s.load_state_dict(torch.load(MODEL_S2S_PATH, map_lo

=== Пример 1 ===
SRC: If you only stay there.
REF: Только бы не вылететь.
S2S: Если ты останешься здесь.
ATT: Если ты оста остаешься там.

=== Пример 2 ===
SRC: I don't know how you do it, Pop, carrying these boxes around every day.
REF: И как ты только справляешься, папа, таская эти коробки взад-вперед целый день.
S2S: Я не знаю, как ты делаешь, но каждый день каждый день.
ATT: Я не знаю, как ты это делаешь, По,, эти этики в каждый день.

=== Пример 3 ===
SRC: We might have a slight edge in mediation.
REF: Возможно, у нас есть небольшое преимущество в переговорах.
S2S: Возможно, мы можем устроить в в в в.
ATT: Возможно, у нас есть небольшая медаль в посредничества.

=== Пример 4 ===
SRC: How long is it going to take you to get him what he needs?
REF: Сколько времени вы будете делать то, что ему нужно?
S2S: Сколько времени тебе нужно это, чтобы он, что?
ATT: Сколько времени тебе нужно, чтобы он его, что?

=== Пример 5 ===
SRC: On 1 April President of the Nagorno Karabagh Republic Bako 

## В целом модели работают, по крайней мере для коротких простых предложений.